In [1]:
import torch


from random import randint

# Data

# Model

In [182]:
from model import ABAE
    
    
# test ABAE model
text = [
    [randint(0,99) for _ in range(randint(10,20))],
    [randint(0,99) for _ in range(randint(10,20))],
    [randint(0,99) for _ in range(randint(10,20))],
    [randint(0,99) for _ in range(randint(10,20))],
    [randint(0,99) for _ in range(randint(10,20))],
    [randint(0,99) for _ in range(randint(10,20))],
    [randint(0,99) for _ in range(randint(10,20))]
]

vocab = set()
for s in text:
    for w in s:
        vocab.add(w)

print(f"sentences: {len(text)}")
print(f"words: {sum([len(s) for s in text])}")
print(f"vocab size: {max(vocab)}")
    
model = ABAE(word_num=max(vocab)+1)
ps = torch.Tensor(text[0]).long()
ns_list = list(map(lambda x: torch.Tensor(x).long(), [text[1], text[2], text[3]]))
rs, ns_list = model(ps, ns_list)

sentences: 7
words: 100
vocab size: 99


# Train

# Test-Inference